# Laboratorio 2

- Jaime Andres Torres Bermejo - 202014866
- Elkin Rafael Cuello - 202215037

## Problema 1: Redes de Transporte

## Planteamiento matemático:

**Conjuntos**
* O: Conjunto de ciudades origen, O = {Bogotá, Medellín}
* D: Conjunto de ciudades destino, D = {Cali, Barranquilla, Pasto, Tunja, Chía, Manizales}

**Parámetros**
* $c_{ij}$: Costo de transporte desde la ciudad origen i hasta la ciudad destino j
* $o_i$: Oferta disponible en la ciudad origen i
* $d_j$: Demanda en la ciudad destino j

**Variable de Decisión**
* $x_{ij}$: Cantidad de productos transportados desde la ciudad origen i hasta la ciudad destino j

**Función Objetivo**
Minimizar la suma de los costos de transporte:

$$
\min \space ( \sum_{i \in O} \sum_{j \in D} c_{ij} x_{ij})
$$

**Restricciones**
* **Restricción de oferta:** no exceder la cantidad disponible en cada origen

$$
\sum_{j \in D} x_{ij} \leq o_i, \forall i \in O
$$

* **Restricción de demanda:** satisfacer la demanda en cada destino

$$
\sum_{i \in O} x_{ij} = d_j, \forall j \in D
$$

In [35]:
from pyomo.environ import *
import math
import numpy as np


# Modelo
model = ConcreteModel()

# Conjuntos
origenes = ['Bogota', 'Medellin']
destinos = ['Cali', 'Barranquilla', 'Pasto', 'Tunja', 'Chia', 'Manizales']
costos = {
    ('Bogota', 'Cali'): 100, ('Medellin', 'Cali'): 2.5,
    ('Bogota', 'Barranquilla'): 2.5, ('Medellin', 'Barranquilla'): 100,
    ('Bogota', 'Pasto'): 1.6, ('Medellin', 'Pasto'): 2.0,
    ('Bogota', 'Tunja'): 1.4, ('Medellin', 'Tunja'): 1.0,
    ('Bogota', 'Chia'): 0.8, ('Medellin', 'Chia'): 1.0,
    ('Bogota', 'Manizales'): 1.4, ('Medellin', 'Manizales'): 0.8
}
demandas = {'Cali': 125, 'Barranquilla': 175, 'Pasto': 225, 'Tunja': 250, 'Chia': 225, 'Manizales': 200}
ofertas = {'Bogota': 550, 'Medellin': 700}

# Variables
model.x = Var(origenes, destinos, within=NonNegativeReals)

# Función objetivo: minimizar los costos de transporte
def objetivo(model):
    return sum(model.x[i, j] * costos[i, j] for i in origenes for j in destinos)
model.objetivo = Objective(rule=objetivo, sense=minimize)

# Restricción de oferta: no exceder la cantidad disponible en cada origen
def oferta_rule(model, i):
    return sum(model.x[i, j] for j in destinos) <= ofertas[i]
model.oferta = Constraint(origenes, rule=oferta_rule)

# Restricción de demanda: satisfacer la demanda en cada destino
def demanda_rule(model, j):
    return sum(model.x[i, j] for i in origenes) == demandas[j]
model.demanda = Constraint(destinos, rule=demanda_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
solver.solve(model)

model.display()

Model unknown

  Variables:
    x : Size=12, Index={Bogota, Medellin}*{Cali, Barranquilla, Pasto, Tunja, Chia, Manizales}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
          ('Bogota', 'Barranquilla') :     0 : 175.0 :  None : False : False : NonNegativeReals
                  ('Bogota', 'Cali') :     0 :   0.0 :  None : False : False : NonNegativeReals
                  ('Bogota', 'Chia') :     0 : 150.0 :  None : False : False : NonNegativeReals
             ('Bogota', 'Manizales') :     0 :   0.0 :  None : False : False : NonNegativeReals
                 ('Bogota', 'Pasto') :     0 : 225.0 :  None : False : False : NonNegativeReals
                 ('Bogota', 'Tunja') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('Medellin', 'Barranquilla') :     0 :   0.0 :  None : False : False : NonNegativeReals
                ('Medellin', 'Cali') :     0 : 125.0 :  None : False : False : NonNegativeReals
                ('Medell

### Analisis de sensibilidad - Problema 1

b. Realize un analísis de Sensibilidad de las cuidades de destino y las cuidades de origen

In [36]:
model.dual = Suffix(direction=Suffix.IMPORT)
resultado = solver.solve(model)

In [37]:
print("\nSolution")

for i in destinos:
    print(f"Dual demanda {i}: {model.dual[model.demanda[i]]}")

for j in origenes:
    print(f"Dual oferta {j}: {model.dual[model.oferta[j]]}")


Solution
Dual demanda Cali: 2.5
Dual demanda Barranquilla: 2.7
Dual demanda Pasto: 1.8
Dual demanda Tunja: 1.0
Dual demanda Chia: 1.0
Dual demanda Manizales: 0.8
Dual oferta Bogota: -0.2
Dual oferta Medellin: 0.0


c. Mueva 50 toneladas de oferta de medellin a bogota y repita el analisis de sensibilidad. ¿Que ha cambiado? Recomendaria este cambio or que otro cambio puede proponer?

Se modifican los parametros

In [38]:
ofertas['Bogota'] = ofertas['Bogota'] + 50
ofertas['Medellin'] = ofertas['Medellin'] - 50 

In [39]:
resultado = solver.solve(model)

In [40]:
print("\nSolution")

for i in destinos:
    print(f"Dual demanda {i}: {model.dual[model.demanda[i]]}")

for j in origenes:
    print(f"Dual oferta {j}: {model.dual[model.oferta[j]]}")


Solution
Dual demanda Cali: 2.5
Dual demanda Barranquilla: 2.7
Dual demanda Pasto: 1.8
Dual demanda Tunja: 1.0
Dual demanda Chia: 1.0
Dual demanda Manizales: 0.8
Dual oferta Bogota: -0.2
Dual oferta Medellin: 0.0


¿Que ha cambiado? Recomendaria este cambio or que otro cambio puede proponer?

RTA: Al realizar este cambio, no se ha modificado ningun valor de el analisis de sensibilidad. La ausencia de cambios en el valor objetivo puede indicar que el problema no es sensible a los parámetros modificados. Esto puede deberse a que los parámetros no influyen directamente en la función objetivo o que sus efectos se cancelan.

# Problema 2: Rutas Óptimas para equipos de Inspección de Infrastructura en Colombia

# Problema 3: Optimización de Sensores en Ciudades Inteligentes

## Planteamiento matemático del problema 3:

### Conjuntos
- **S**: Conjunto de sensores disponibles.
- **L**: Conjunto de localidades que necesitan ser monitoreadas.

### Parámetros
- **EC<sub>s</sub>**: Costo de consumo de energía del sensor *s*.
- **CC<sub>s,l</sub>**: Costo de comunicación entre el sensor *s* y la localidad *l*.
- **IC<sub>l</sub>**: Costo de instalación en la localidad *l*.
- **SC<sub>s,l</sub>**: Cobertura del sensor *s* en la localidad *l* (1 si el sensor puede cubrir la localidad, 0 en caso contrario).
- **ZC<sub>l,l'</sub>**: Cobertura de zona entre las localidades *l* y *l'* (1 si las localidades son adyacentes, 0 en caso contrario).

### Variables de Decisión
- **x<sub>s,l</sub>**: Variable binaria que indica si el sensor *s* está instalado en la localidad *l* (1 si está instalado, 0 en caso contrario).
- **y<sub>s</sub>**: Variable binaria que indica si el sensor *s* está instalado en cualquier localidad (1 si está instalado, 0 en caso contrario).

### Función Objetivo
Minimizar el costo total:

$$
\min \space ( \sum_{s \in S} EC_{s} \cdot y_{s} + \sum_{s \in S} \sum_{l \in L} CC_{s,l} \cdot x_{s,l} + \sum_{s \in S} \sum_{l \in L} IC_{l} \cdot x_{s,l})
$$

### Restricciones
#### Cobertura de localidades:
$$
\sum_{loc\_adj \in L} x_{s, loc\_adj} \geq 1 \quad \forall s \in S, \forall l \in L | ZC_{l, loc\_adj} = 1
$$

Esta restricción garantiza que cada localidad sea cubierta por al menos un sensor.


### Corriendo el programa sin modificaciones

Corra el modelo sin modificaciónes, evalue los resultados

In [41]:
from pyomo.environ import *
import random

# Crear el modelo
model = ConcreteModel()

# Conjuntos
locations = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12']
sensors = ['S1', 'S2', 'S3']

# Parámetros

# Costo de consumo de energía de los sensores
energy_consumption = {'S1': 7, 'S2': 4, 'S3': 8}

# Costo de comunicación entre los sensores y las localidades
communication_cost = {
    ('L1', 'S1'): 48, ('L2', 'S1'): 38, ('L3', 'S1'): 24, ('L4', 'S1'): 17,
    ('L5', 'S1'): 30, ('L6', 'S1'): 48, ('L7', 'S1'): 28, ('L8', 'S1'): 32,
    ('L9', 'S1'): 20, ('L10', 'S1'): 20, ('L11', 'S1'): 33, ('L12', 'S1'): 45,
    ('L1', 'S2'): 49, ('L2', 'S2'): 33, ('L3', 'S2'): 12, ('L4', 'S2'): 31,
    ('L5', 'S2'): 11, ('L6', 'S2'): 33, ('L7', 'S2'): 39, ('L8', 'S2'): 47,
    ('L9', 'S2'): 11, ('L10', 'S2'): 30, ('L11', 'S2'): 42, ('L12', 'S2'): 21,
    ('L1', 'S3'): 31, ('L2', 'S3'): 34, ('L3', 'S3'): 36, ('L4', 'S3'): 37,
    ('L5', 'S3'): 25, ('L6', 'S3'): 24, ('L7', 'S3'): 12, ('L8', 'S3'): 46,
    ('L9', 'S3'): 16, ('L10', 'S3'): 30, ('L11', 'S3'): 18, ('L12', 'S3'): 48
}

# Costo de instalación en cada localidad
installation_cost = {
    'L1': 250, 'L2': 100, 'L3': 200, 'L4': 250, 'L5': 300, 'L6': 120,
    'L7': 170, 'L8': 150, 'L9': 270, 'L10': 130, 'L11': 100, 'L12': 230
}

# Cobertura de sensores en cada localidad
sensor_coverage = {
    ('L1', 'S1'): 1, ('L1', 'S2'): 0, ('L1', 'S3'): 1,
    ('L2', 'S1'): 1, ('L2', 'S2'): 0, ('L2', 'S3'): 1,
    ('L3', 'S1'): 1, ('L3', 'S2'): 0, ('L3', 'S3'): 1,
    ('L4', 'S1'): 1, ('L4', 'S2'): 1, ('L4', 'S3'): 0,
    ('L5', 'S1'): 1, ('L5', 'S2'): 1, ('L5', 'S3'): 1,
    ('L6', 'S1'): 1, ('L6', 'S2'): 1, ('L6', 'S3'): 1,
    ('L7', 'S1'): 1, ('L7', 'S2'): 0, ('L7', 'S3'): 1,
    ('L8', 'S1'): 1, ('L8', 'S2'): 1, ('L8', 'S3'): 1,
    ('L9', 'S1'): 1, ('L9', 'S2'): 1, ('L9', 'S3'): 1,
    ('L10', 'S1'): 1, ('L10', 'S2'): 1, ('L10', 'S3'): 0,
    ('L11', 'S1'): 1, ('L11', 'S2'): 1, ('L11', 'S3'): 1,
    ('L12', 'S1'): 1, ('L12', 'S2'): 0, ('L12', 'S3'): 1
}


zone_coverage = {
    ('L1', 'L1'): 1, ('L1', 'L2'): 1, ('L1', 'L3'): 1, ('L1', 'L4'): 0, ('L1', 'L5'): 1, ('L1', 'L6'): 0, ('L1', 'L7'): 0, ('L1', 'L8'): 0, ('L1', 'L9'): 0, ('L1', 'L10'): 0, ('L1', 'L11'): 0, ('L1', 'L12'): 0,
    ('L2', 'L1'): 1, ('L2', 'L2'): 1, ('L2', 'L3'): 0, ('L2', 'L4'): 0, ('L2', 'L5'): 0, ('L2', 'L6'): 0, ('L2', 'L7'): 0, ('L2', 'L8'): 0, ('L2', 'L9'): 0, ('L2', 'L10'): 0, ('L2', 'L11'): 0, ('L2', 'L12'): 0,
    ('L3', 'L1'): 1, ('L3', 'L2'): 0, ('L3', 'L3'): 1, ('L3', 'L4'): 1, ('L3', 'L5'): 1, ('L3', 'L6'): 1, ('L3', 'L7'): 1, ('L3', 'L8'): 1, ('L3', 'L9'): 0, ('L3', 'L10'): 0, ('L3', 'L11'): 0, ('L3', 'L12'): 0,
    ('L4', 'L1'): 0, ('L4', 'L2'): 0, ('L4', 'L3'): 1, ('L4', 'L4'): 1, ('L4', 'L5'): 1, ('L4', 'L6'): 1, ('L4', 'L7'): 0, ('L4', 'L8'): 1, ('L4', 'L9'): 0, ('L4', 'L10'): 0, ('L4', 'L11'): 1, ('L4', 'L12'): 0,
    ('L5', 'L1'): 1, ('L5', 'L2'): 1, ('L5', 'L3'): 1, ('L5', 'L4'): 1, ('L5', 'L5'): 1, ('L5', 'L6'): 0, ('L5', 'L7'): 0, ('L5', 'L8'): 0, ('L5', 'L9'): 0, ('L5', 'L10'): 1, ('L5', 'L11'): 1, ('L5', 'L12'): 0,
    ('L6', 'L1'): 0, ('L6', 'L2'): 0, ('L6', 'L3'): 1, ('L6', 'L4'): 1, ('L6', 'L5'): 0, ('L6', 'L6'): 1, ('L6', 'L7'): 0, ('L6', 'L8'): 1, ('L6', 'L9'): 0, ('L6', 'L10'): 0, ('L6', 'L11'): 1, ('L6', 'L12'): 0,
    ('L7', 'L1'): 0, ('L7', 'L2'): 0, ('L7', 'L3'): 1, ('L7', 'L4'): 0, ('L7', 'L5'): 0, ('L7', 'L6'): 0, ('L7', 'L7'): 1, ('L7', 'L8'): 1, ('L7', 'L9'): 0, ('L7', 'L10'): 0, ('L7', 'L11'): 0, ('L7', 'L12'): 1,
    ('L8', 'L1'): 0, ('L8', 'L2'): 0, ('L8', 'L3'): 0, ('L8', 'L4'): 1, ('L8', 'L5'): 0, ('L8', 'L6'): 1, ('L8', 'L7'): 1, ('L8', 'L8'): 1, ('L8', 'L9'): 1, ('L8', 'L10'): 0, ('L8', 'L11'): 1, ('L8', 'L12'): 1,
    ('L9', 'L1'): 0, ('L9', 'L2'): 0, ('L9', 'L3'): 0, ('L9', 'L4'): 0, ('L9', 'L5'): 0, ('L9', 'L6'): 0, ('L9', 'L7'): 0, ('L9', 'L8'): 1, ('L9', 'L9'): 1, ('L9', 'L10'): 1, ('L9', 'L11'): 1, ('L9', 'L12'): 1,
    ('L10', 'L1'): 0, ('L10', 'L2'): 0, ('L10', 'L3'): 0, ('L10', 'L4'): 0, ('L10', 'L5'): 1, ('L10', 'L6'): 0, ('L10', 'L7'): 0, ('L10', 'L8'): 0, ('L10', 'L9'): 1, ('L10', 'L10'): 1, ('L10', 'L11'): 1, ('L10', 'L12'): 0,
    ('L11', 'L1'): 0, ('L11', 'L2'): 0, ('L11', 'L3'): 0, ('L11', 'L4'): 1, ('L11', 'L5'): 1, ('L11', 'L6'): 1, ('L11', 'L7'): 0, ('L11', 'L8'): 1, ('L11', 'L9'): 1, ('L11', 'L10'): 1, ('L11', 'L11'): 1, ('L11', 'L12'): 0,
    ('L12', 'L1'): 0, ('L12', 'L2'): 0, ('L12', 'L3'): 0, ('L12', 'L4'): 0, ('L12', 'L5'): 0, ('L12', 'L6'): 0, ('L12', 'L7'): 1, ('L12', 'L8'): 1, ('L12', 'L9'): 1, ('L12', 'L10'): 0, ('L12', 'L11'): 0, ('L12', 'L12'): 1,
}



# Variables de Decisión
model.x = Var(sensors, locations, domain=Binary)  # Si un sensor está instalado en una ubicación
model.y = Var(sensors, domain=Binary)  # Si un sensor está instalado en cualquier ubicación

# Función Objetivo: Minimizar el costo total
def objective_rule(model):
    return (
        sum(energy_consumption[s] * model.y[s] for s in sensors) +  # Costo de energía
        sum(communication_cost[(l, s)] * model.x[s, l] for s in sensors for l in locations) +  # Costo de comunicación
        sum(installation_cost[l] * model.x[s, l] for s in sensors for l in locations)  # Costo de instalación
    )
model.obj = Objective(rule=objective_rule, sense=minimize)


# Restricciones
def coverage_rule(model, s, l):
    if sensor_coverage[(l, s)] == 1:
        return sum(model.x[s, loc_adj] for loc_adj in locations if zone_coverage[(l, loc_adj)] == 1) >= 1
    else:
        return Constraint.Skip
model.coverage_constraint = Constraint(sensors, locations, rule=coverage_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
results = solver.solve(model)

# Mostrar resultados
model.display()

Model unknown

  Variables:
    x : Size=36, Index={S1, S2, S3}*{L1, L2, L3, L4, L5, L6, L7, L8, L9, L10, L11, L12}
        Key           : Lower : Value : Upper : Fixed : Stale : Domain
         ('S1', 'L1') :     0 :   0.0 :     1 : False : False : Binary
        ('S1', 'L10') :     0 :   0.0 :     1 : False : False : Binary
        ('S1', 'L11') :     0 :   1.0 :     1 : False : False : Binary
        ('S1', 'L12') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L2') :     0 :   1.0 :     1 : False : False : Binary
         ('S1', 'L3') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L4') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L5') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L6') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L7') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L8') :     0 :   1.0 :     1 : False : False : Binary
         ('S1', 'L9') :     0 : 

Evalue los resultados:

El programa minimiza correctamente el costo de cubrir todas las zonas propuestas. Esto se puede verificar haciendo el proceso a mano para cada sensor, y teniendo en cuenta el sensor_coverage.

### Corriendo el programa, teniendo en cuenta solo el costo de instalación

Corra el modelo unicamente teniendo en cuenta el costo de instalación en la función objetivo. ¿Como cambia la solución?

Para esto se modifica la función objetivo, de forma que queda de la siguiente manera:

$$
\min \space ( \sum_{l \in L} IC_{l} \cdot x_{s,l})
$$

In [42]:
from pyomo.environ import *
import random

# Crear el modelo
model = ConcreteModel()

# Conjuntos
locations = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12']
sensors = ['S1', 'S2', 'S3']

# Parámetros

# Costo de consumo de energía de los sensores
energy_consumption = {'S1': 7, 'S2': 4, 'S3': 8}

# Costo de comunicación entre los sensores y las localidades
communication_cost = {
    ('L1', 'S1'): 48, ('L2', 'S1'): 38, ('L3', 'S1'): 24, ('L4', 'S1'): 17,
    ('L5', 'S1'): 30, ('L6', 'S1'): 48, ('L7', 'S1'): 28, ('L8', 'S1'): 32,
    ('L9', 'S1'): 20, ('L10', 'S1'): 20, ('L11', 'S1'): 33, ('L12', 'S1'): 45,
    ('L1', 'S2'): 49, ('L2', 'S2'): 33, ('L3', 'S2'): 12, ('L4', 'S2'): 31,
    ('L5', 'S2'): 11, ('L6', 'S2'): 33, ('L7', 'S2'): 39, ('L8', 'S2'): 47,
    ('L9', 'S2'): 11, ('L10', 'S2'): 30, ('L11', 'S2'): 42, ('L12', 'S2'): 21,
    ('L1', 'S3'): 31, ('L2', 'S3'): 34, ('L3', 'S3'): 36, ('L4', 'S3'): 37,
    ('L5', 'S3'): 25, ('L6', 'S3'): 24, ('L7', 'S3'): 12, ('L8', 'S3'): 46,
    ('L9', 'S3'): 16, ('L10', 'S3'): 30, ('L11', 'S3'): 18, ('L12', 'S3'): 48
}

# Costo de instalación en cada localidad
installation_cost = {
    'L1': 250, 'L2': 100, 'L3': 200, 'L4': 250, 'L5': 300, 'L6': 120,
    'L7': 170, 'L8': 150, 'L9': 270, 'L10': 130, 'L11': 100, 'L12': 230
}

# Cobertura de sensores en cada localidad
sensor_coverage = {
    ('L1', 'S1'): 1, ('L1', 'S2'): 0, ('L1', 'S3'): 1,
    ('L2', 'S1'): 1, ('L2', 'S2'): 0, ('L2', 'S3'): 1,
    ('L3', 'S1'): 1, ('L3', 'S2'): 0, ('L3', 'S3'): 1,
    ('L4', 'S1'): 1, ('L4', 'S2'): 1, ('L4', 'S3'): 0,
    ('L5', 'S1'): 1, ('L5', 'S2'): 1, ('L5', 'S3'): 1,
    ('L6', 'S1'): 1, ('L6', 'S2'): 1, ('L6', 'S3'): 1,
    ('L7', 'S1'): 1, ('L7', 'S2'): 0, ('L7', 'S3'): 1,
    ('L8', 'S1'): 1, ('L8', 'S2'): 1, ('L8', 'S3'): 1,
    ('L9', 'S1'): 1, ('L9', 'S2'): 1, ('L9', 'S3'): 1,
    ('L10', 'S1'): 1, ('L10', 'S2'): 1, ('L10', 'S3'): 0,
    ('L11', 'S1'): 1, ('L11', 'S2'): 1, ('L11', 'S3'): 1,
    ('L12', 'S1'): 1, ('L12', 'S2'): 0, ('L12', 'S3'): 1
}


zone_coverage = {
    ('L1', 'L1'): 1, ('L1', 'L2'): 1, ('L1', 'L3'): 1, ('L1', 'L4'): 0, ('L1', 'L5'): 1, ('L1', 'L6'): 0, ('L1', 'L7'): 0, ('L1', 'L8'): 0, ('L1', 'L9'): 0, ('L1', 'L10'): 0, ('L1', 'L11'): 0, ('L1', 'L12'): 0,
    ('L2', 'L1'): 1, ('L2', 'L2'): 1, ('L2', 'L3'): 0, ('L2', 'L4'): 0, ('L2', 'L5'): 0, ('L2', 'L6'): 0, ('L2', 'L7'): 0, ('L2', 'L8'): 0, ('L2', 'L9'): 0, ('L2', 'L10'): 0, ('L2', 'L11'): 0, ('L2', 'L12'): 0,
    ('L3', 'L1'): 1, ('L3', 'L2'): 0, ('L3', 'L3'): 1, ('L3', 'L4'): 1, ('L3', 'L5'): 1, ('L3', 'L6'): 1, ('L3', 'L7'): 1, ('L3', 'L8'): 1, ('L3', 'L9'): 0, ('L3', 'L10'): 0, ('L3', 'L11'): 0, ('L3', 'L12'): 0,
    ('L4', 'L1'): 0, ('L4', 'L2'): 0, ('L4', 'L3'): 1, ('L4', 'L4'): 1, ('L4', 'L5'): 1, ('L4', 'L6'): 1, ('L4', 'L7'): 0, ('L4', 'L8'): 1, ('L4', 'L9'): 0, ('L4', 'L10'): 0, ('L4', 'L11'): 1, ('L4', 'L12'): 0,
    ('L5', 'L1'): 1, ('L5', 'L2'): 1, ('L5', 'L3'): 1, ('L5', 'L4'): 1, ('L5', 'L5'): 1, ('L5', 'L6'): 0, ('L5', 'L7'): 0, ('L5', 'L8'): 0, ('L5', 'L9'): 0, ('L5', 'L10'): 1, ('L5', 'L11'): 1, ('L5', 'L12'): 0,
    ('L6', 'L1'): 0, ('L6', 'L2'): 0, ('L6', 'L3'): 1, ('L6', 'L4'): 1, ('L6', 'L5'): 0, ('L6', 'L6'): 1, ('L6', 'L7'): 0, ('L6', 'L8'): 1, ('L6', 'L9'): 0, ('L6', 'L10'): 0, ('L6', 'L11'): 1, ('L6', 'L12'): 0,
    ('L7', 'L1'): 0, ('L7', 'L2'): 0, ('L7', 'L3'): 1, ('L7', 'L4'): 0, ('L7', 'L5'): 0, ('L7', 'L6'): 0, ('L7', 'L7'): 1, ('L7', 'L8'): 1, ('L7', 'L9'): 0, ('L7', 'L10'): 0, ('L7', 'L11'): 0, ('L7', 'L12'): 1,
    ('L8', 'L1'): 0, ('L8', 'L2'): 0, ('L8', 'L3'): 0, ('L8', 'L4'): 1, ('L8', 'L5'): 0, ('L8', 'L6'): 1, ('L8', 'L7'): 1, ('L8', 'L8'): 1, ('L8', 'L9'): 1, ('L8', 'L10'): 0, ('L8', 'L11'): 1, ('L8', 'L12'): 1,
    ('L9', 'L1'): 0, ('L9', 'L2'): 0, ('L9', 'L3'): 0, ('L9', 'L4'): 0, ('L9', 'L5'): 0, ('L9', 'L6'): 0, ('L9', 'L7'): 0, ('L9', 'L8'): 1, ('L9', 'L9'): 1, ('L9', 'L10'): 1, ('L9', 'L11'): 1, ('L9', 'L12'): 1,
    ('L10', 'L1'): 0, ('L10', 'L2'): 0, ('L10', 'L3'): 0, ('L10', 'L4'): 0, ('L10', 'L5'): 1, ('L10', 'L6'): 0, ('L10', 'L7'): 0, ('L10', 'L8'): 0, ('L10', 'L9'): 1, ('L10', 'L10'): 1, ('L10', 'L11'): 1, ('L10', 'L12'): 0,
    ('L11', 'L1'): 0, ('L11', 'L2'): 0, ('L11', 'L3'): 0, ('L11', 'L4'): 1, ('L11', 'L5'): 1, ('L11', 'L6'): 1, ('L11', 'L7'): 0, ('L11', 'L8'): 1, ('L11', 'L9'): 1, ('L11', 'L10'): 1, ('L11', 'L11'): 1, ('L11', 'L12'): 0,
    ('L12', 'L1'): 0, ('L12', 'L2'): 0, ('L12', 'L3'): 0, ('L12', 'L4'): 0, ('L12', 'L5'): 0, ('L12', 'L6'): 0, ('L12', 'L7'): 1, ('L12', 'L8'): 1, ('L12', 'L9'): 1, ('L12', 'L10'): 0, ('L12', 'L11'): 0, ('L12', 'L12'): 1,
}



# Variables de Decisión
model.x = Var(sensors, locations, domain=Binary)  # Si un sensor está instalado en una ubicación
model.y = Var(sensors, domain=Binary)  # Si un sensor está instalado en cualquier ubicación

# Función Objetivo: Minimizar el costo total
def objective_rule(model):
    return (
        sum(installation_cost[l] * model.x[s, l] for s in sensors for l in locations)  # Aquí solo se tiene en cuenta el costo de instalación
    )
model.obj = Objective(rule=objective_rule, sense=minimize)


# Restricciones
def coverage_rule(model, s, l):
    if sensor_coverage[(l, s)] == 1:
        return sum(model.x[s, loc_adj] for loc_adj in locations if zone_coverage[(l, loc_adj)] == 1) >= 1
    else:
        return Constraint.Skip
model.coverage_constraint = Constraint(sensors, locations, rule=coverage_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
results = solver.solve(model)

# Mostrar resultados
model.display()

Model unknown

  Variables:
    x : Size=36, Index={S1, S2, S3}*{L1, L2, L3, L4, L5, L6, L7, L8, L9, L10, L11, L12}
        Key           : Lower : Value : Upper : Fixed : Stale : Domain
         ('S1', 'L1') :     0 :   0.0 :     1 : False : False : Binary
        ('S1', 'L10') :     0 :   0.0 :     1 : False : False : Binary
        ('S1', 'L11') :     0 :   1.0 :     1 : False : False : Binary
        ('S1', 'L12') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L2') :     0 :   1.0 :     1 : False : False : Binary
         ('S1', 'L3') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L4') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L5') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L6') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L7') :     0 :   0.0 :     1 : False : False : Binary
         ('S1', 'L8') :     0 :   1.0 :     1 : False : False : Binary
         ('S1', 'L9') :     0 : 

¿Como cambia la solución?

Al correr el programa teniendo en cuenta unicamente el costo de instalación en la función objetivo, se obtiene el mismo resultado. Esto tiene sentido, debido a que los costos de instalación son los mas altos respecto a los demás costos. En otras palabras, es el costo dominante que determina el comportamiento de la función objetivo.